<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/LLM/training/examples_fine_tune_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pub.towardsai.net/fine-tuning-llms-from-zero-to-hero-with-python-ollama-52258966bb6d

In [1]:

!pip install unsloth
# !pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes transformers datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


Load the basemodel in unsloth

In [2]:
from unsloth import FastLanguageModel

model_name = "unsloth/phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Adjust based on your data length
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Add LoRA Adapter to the layers for efficient training

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - higher = more parameters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


The training dataset that we read in looks like:

[
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
]

In [4]:
import json
from datasets import Dataset

with open("training_data.json", "r") as f:
    data = json.load(f)
# Format for training
def format_chat_template(item):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": item['input']},
            {"role": "assistant", "content": item['output']}
        ],
        tokenize=False,
        add_generation_prompt=False
    )
# Create the training dataset
formatted_data = [{"text": format_chat_template(item)} for item in data]
dataset = Dataset.from_list(formatted_data)
# Check what it looks like
print("Sample training example:")
print(formatted_data[0]["text"])

Sample training example:
<|user|>
classify the following log event sequence as normal or suspicious
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:22:26,User authentication successful,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:23:26,User logout successful,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:28:26,Database query executed,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:32:26,Connection established,DEBUG
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:38:26,Database query executed,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:52:26,User logout successful,INFO<|end|>
<|assistant|>
normal<|end|>
<|endoftext|>


We are ready to go, so start the training !

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

training_steps=60

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=training_steps,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",  # Use "adamw_torch" if you get optimizer errors
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=30,
    ),
)
# Start training! 🚀
trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/70 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 70 | Num Epochs = 7 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erik-bamberg (erik-bamberg-self-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


OutOfResources: out of resource: shared memory, Required: 98304, Hardware limit: 65536. Reducing block sizes or `num_stages` may help.

SAVE our fine-tuned model in the GGUF format which is compatible to Ollama.

In [ ]:
# save_pretrained_gguf is a unsloth function. this is ot available on standard hugging face models
model.save_pretrained_gguf(
    "fine_tuned_model",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of size/quality
)

In [ ]:
%%writefile Modelfile.py
FROM ./unsloth.Q4_K_M.gguf
PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER stop ["<|endoftext|>"]
TEMPLATE "{{ .Prompt }}"
SYSTEM "You are a specialized C64 Basic coder assistant."

you can use ollama to create a model from the modelfile:

ollama create \<modelname\> -f Modelfile